## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 17
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_17


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
%%time

X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 16.8 s


,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
0,Tampa Bay Buccaneers,16,2021,29.4,7.3,306.6,100.3,10.9,99.2,21.0,...,9.9,91.4,17.0625,13.2500,13.2500,1.3750,1.873529,0.104294,2.5000,3.333333
1,Dallas Cowboys,16,2021,29.9,7.5,281.0,101.7,11.0,121.8,20.8,...,12.1,110.6,17.1875,13.4375,12.8125,1.4375,1.459596,0.114754,2.1875,3.181818


In [5]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [6]:
X['games_played'].unique().tolist()

[16]

In [7]:
# X[X['games_played'] == 8]

In [8]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
5,Los Angeles Rams,2021,week_17,0.227658,0.158052,0.180777,0.245732,0.203055
0,Tampa Bay Buccaneers,2021,week_17,0.224782,0.158080,0.145923,0.221369,0.187539
8,Cincinnati Bengals,2021,week_17,0.195089,0.107209,0.193160,0.226843,0.180575


In [9]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [10]:
predictions

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
5,Los Angeles Rams,2021,week_17,0.227658,0.158052,0.180777,0.245732,0.203055
0,Tampa Bay Buccaneers,2021,week_17,0.224782,0.158080,0.145923,0.221369,0.187539
8,Cincinnati Bengals,2021,week_17,0.195089,0.107209,0.193160,0.226843,0.180575
9,San Francisco 49ers,2021,week_17,0.166961,0.105340,0.179985,0.238425,0.172678
1,Dallas Cowboys,2021,week_17,0.210965,0.143178,0.107692,0.194202,0.164009
4,Buffalo Bills,2021,week_17,0.207130,0.102638,0.144117,0.196772,0.162664
2,Kansas City Chiefs,2021,week_17,0.186892,0.095725,0.094981,0.196295,0.143474
15,New England Patriots,2021,week_17,0.130987,0.096923,0.081862,0.195213,0.126246
11,Green Bay Packers,2021,week_17,0.117059,0.055616,0.070238,0.211386,0.113575
3,Los Angeles Chargers,2021,week_17,0.121515,0.065687,0.092854,0.166799,0.111714
